In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import subprocess, time, psutil
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from utils import SOHTransformer, load_and_proc_data, monitor_idle_gpu_cpu, train_model, evaluate_model, SOHDataset

In [2]:
import os
directory = "data/battery_scaled"
file_list = sorted([directory+'/'+f for f in os.listdir(directory) if f.endswith(".csv")])
targets = ['available_capacity (Ah)']

SEQ_LEN = 100
BATCH_SIZE = 32
features = ['soc', 'pack_voltage (V)', 'charge_current (A)', 'max_temperature (℃)', 'min_temperature (℃)', 'available_capacity (Ah)']
NUM_FEATURES = len(features)

data = torch.load('test_dataset.pt')
X_loaded = data['X']
y_loaded = data['y']

# Recreate dataset and dataloader
test_dataset = SOHDataset(X_loaded, y_loaded)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SOHTransformer(input_dim=NUM_FEATURES, embed_dim=256).to(device)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)

evaluate_model(model, test_loader, "models/best_TRANSFORMER_SUNUM.pth", 'outputs/error_results_TRANSFORMER_SUNUM.txt', 'transformer_SUNUM', plot_fig = False, device=device)


(315489, 100)
(315489, 100)

Test RMSE: 0.0576
Test MAE: 0.0476
Test R²: -2.0817
Test MAPE: 5.02%
Test PCC: [0.27528563 0.20793125 0.24225906 0.3450083  0.21802141 0.37699237
 0.17546828 0.26003024 0.18918964 0.14990918 0.42578757 0.30090287
 0.27320704 0.20488797 0.30934715 0.22497496 0.22929202 0.06072526
 0.21936087 0.3386822  0.19494925 0.325458   0.21058853 0.22069784
 0.22903699 0.2425692  0.29556462 0.20305851 0.18625961 0.24783058
 0.2502866  0.25450507 0.11561643 0.2581497  0.23352452 0.36990076
 0.34495425 0.23090436 0.18514085 0.2181188  0.20306647 0.32639265
 0.16045551 0.24342038 0.2003804  0.28882945 0.20917036 0.25154078
 0.28746894 0.35607427 0.19362657 0.30212557 0.22896296 0.25328755
 0.18837245 0.25202653 0.29953194 0.22039646 0.32010978 0.16916154
 0.22987683 0.1310169  0.2568001  0.29352695 0.22615981 0.21167281
 0.28567585 0.26155412 0.39076898 0.21598808 0.20822525 0.3189998
 0.3242127  0.21438614 0.34488532 0.26006946 0.1609829  0.31486255
 0.28398195 0.14102611